# Automobile Sales Trends and Recession Insights Dashboard

## Overview Summary
You are building a **Dash dashboard** that dynamically visualizes automobile sales data using interactive dropdowns and multiple chart types. The app has two main reporting modes:

**1. Yearly Automobile Sales Statistics**

In this mode, the user selects a specific year and sees:
- 📈 Line chart: Average automobile sales per year.
- 📊 Line chart: Monthly automobile sales for that year.
- 📊 Bar chart: Average vehicles sold by vehicle type.
- 🥧 Pie chart: Total advertisement expenditure by vehicle type.

**2. Recession Period Statistics**

In this mode, year selection is disabled. It shows:
- 📈 Line chart: Sales fluctuation during recession years.
- 📊 Bar chart: Avg. number of vehicles sold during recessions by type.
- 🥧 Pie chart: Ad expenditure share by type during recessions.
- 📊 Bar chart: Unemployment rate effect on sales by vehicle type.


## Dataset Key Variables
`Automobile_Sales`: Target value for most charts.

`Recession`: 1 (yes) or 0 (no) – for filtering data during recessions.

`Vehicle_Type`: Categorical variable with values like *Supperminicar*, *Sports*, *Executivecar*, etc.

`unemployment_rate`: Used in correlation/unemployment effect charts.

`Seasonality_Weight`, `Consumer_Confidence`, `Price`, etc., are available for more advanced visualizations.

## Dashboard Technical Setup
✅ **Requirements**
- Two dropdown menus:
    - 1st for **Report Type** (`Yearly` or `Recession`).
    - 2nd for **Year** – only enabled when `Yearly` is selected.
- App layout includes:
    - One outer division → Two inner divisions:
    - One for input (dropdowns),
    - One for output (charts).
- Callback:
    - Controls enabling/disabling year dropdown.
    - Triggers graph updates when selection changes.
- Display: 4 graphs, 2 rows × 2 columns.


In [42]:
import dash
import more_itertools
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

In [17]:
# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')


### **TASK 2.1: Create a Dash application and give it a meaningful title** 
- Provide title of the dash application title as `Automobile Sales Statistics Dashboard`
- Make the heading center aligned
- set color as `#503D36`
- font size as `24`

### **TASK 2.2: Add drop-down menus to your dashboard with appropriate titles and options**
Parameters to be updated in `dcc.Dropdown`:
- Set *id* to `dropdown-statistics`.
- Set *options* to a **list containing dictionaries** with label and user-provided values for value.

**Option List Parameters**

**1st option:**
- label: `Yearly Statistics`
- value: `Yearly Statistics`

**2nd option:**
- label: `Recession Period Statistics`
- value: `Recession Period Statistics`


Set *placeholder* to` Select a report type`.

Set *value* to `Select Statistics`.

#### Select Year Dropdown
- You can pass a list of years as options in this dropdowwn [hint: year_list = [i for i in range(1980, 2024, 1)]]
- Set *id* to `select-year`
- Set *value* to `Select-year`
- Set *placeholder* to `Select-year`

**OPTIONAL: Set style to include width as 80%, padding as 3px, font size as 20px, and text-align-last as center.** 

### **TASK 2.3: Add a division for output display with appropriate id and classname property**
- Set *id* to `output-container`
- Set *className* to `chart-grid`
- Set *style* to be displayed as a `flex`

In [44]:
# Initialize the Dash app
app = dash.Dash(__name__)

# List of years 
year_list = [i for i in range(1980, 2024, 1)]

# Create the layout of the app
app.layout = html.Div([
    # TASK 2.1: Add title to the dashboard
    html.H1("Automobile Sales Statistics Dashboard", 
            style={'textAlign': 'center', 'color': '#583D36', 'font-size': 24}),

    # TASK 2.2: Dropdown for selecting report type
    html.Div([
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=[
                {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
            ],
            value='Select Statistics',
            placeholder='Select a report type',
            style={'width': '80%', 'padding': '3px', 'fontSize': 20, 'textAlign': 'center'}
        )
    ]),

    # Dropdown for selecting year
    html.Div(dcc.Dropdown(
        id='select-year',
        options=[{'label': i, 'value': i} for i in year_list],
        value='Select-year',
        placeholder='Select year',
        style={'width': '80%', 'padding': '3px', 'fontSize': 20, 'textAlign': 'center'}
    )),

    # TASK 2.3: Add a division for output display
    html.Div([
        html.Div(id='output-container', className='chart-grid', style={'display': 'flex'}),
    ])
])

### **TASK 2.4: Creating Callbacks; Define the callback function to update the input container based on the selected statistics and the output container**

#### Update Input Container callback function
The purpose of this function is to enable or disable the year selection dropdown based on the user's choice of report type from another dropdown.Here we are having 2 parts:

#### Callback definition:
The parameters of the Callback definition are as follows:
**Output**: Specifies the output of the callback, which in this case is the `disabled` property of the dropdown with the ID `select-year`.
**Input**: Specifies the input that triggers the callback, which is the `value` property of the dropdown with the ID `dropdown-statistics`.

#### Callback Function:
- The function checks the value of selected statistics. If the selected statistics is `Yearly Statistics` the function returns False. This means the `disabled` property of the select-year dropdown will be set to False, enabling the dropdown so the user can select a year.
- If selected statistics is not `Yearly Statistics` (i.e., it is `Recession Period Statistics`), the function returns True. This means the `disabled` property of the select-year dropdown will be set to True, disabling the dropdown as selecting a year is not relevant in this context.


In [47]:
# TASK 2.4: Creating Callbacks

# Input Container
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics', component_property='value'))

def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True

#### Update Output Container callback function
Our layout has 1 output container, and we will be required to return the plots developed `dcc.Graph()` into this container `as divisions`.
For each report we need to display four plots.

The `update_output_container` function in this Dash application dynamically generates and displays graphs based on the user's selection from the dropdown menus. Specifically, it creates different sets of graphs for `Recession Period Statistics` and `Yearly Statistics`. We will be creating 4 plots for each report.

Here we have 2 parts:

#### Callback Definition:
Parameters:
- **Output**: This specifies that the `children` property of the component with the ID `output-container` will be updated.In Dash, children typically refers to the content within a container, such as graphs, text, or other HTML elements.
- **Input**:
    - **Input Property for Report type**: This specifies that the `value` property of the dropdown menu with the ID `dropdown-statistics` (which allows the user to select between `Yearly Statistics` and `Recession Period Statistics`) will trigger the callback when it changes.
    - **Input property for year selection**: This specifies that the `value` property of the dropdown menu with the ID `select-year` (which allows the user to select a specific year) will trigger the callback when it changes.

#### Callback Function:
Here you will filter the data based on the `report type` selected.

When `Recession Period Statistics` is selected, the data is filtered to include only recession periods where `Recession equals 1`. Conversely, when `Yearly Statistics` is chosen, the data is filtered based on the selected year.


In [50]:
# Output Container
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'),
     Input(component_id='select-year', component_property='value')])

def update_output_container(selected_statistics, input_year):
    if selected_statistics == 'Recession Period Statistics':
        recession_data = data[data['Recession'] == 1]

        #Plot 1： Automobile sales fluctuate over Recession Period (year wise) using line chart
            # grouping data for plotting
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        
        # Plotting the line graph
        R_chart1 = dcc.Graph(figure=px.line(yearly_rec, x='Year', y='Automobile_Sales',
                                            title="Average Automobile Sales fluctuation over Recession Period"))

        #Plot 2： Calculate the average number of vehicles sold by vehicle type and represent as a Bar chart
            # Use groupby to create relevant data for plotting. 
            # Hint: Use Vehicle_Type and Automobile_Sales columns
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        
        R_chart2 = dcc.Graph(figure=px.bar(average_sales, x='Vehicle_Type', y='Automobile_Sales',
                                           title="Average Number of Vehicles Sold by Vehicle Type"))

        # Plot 3: Pie chart for total expenditure share by vehicle type during recessions
            # grouping data for plotting
            # Hint: Use Vehicle_Type and Advertising_Expenditure columns
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        
        R_chart3 = dcc.Graph(figure=px.pie(exp_rec, values='Advertising_Expenditure', names='Vehicle_Type',
                                           title='Total Expenditure Share by Vehicle Type During Recession'))

        # Plot 4： Develop a Bar chart for the effect of unemployment rate on vehicle type and sales
            # grouping data for plotting
            # Hint: Use unemployment_rate,Vehicle_Type and Automobile_Sales columns
        unemp_data = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        
        R_chart4 = dcc.Graph(figure=px.bar(unemp_data, x='unemployment_rate', y='Automobile_Sales',
                                           color='Vehicle_Type',
                                           labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
                                           title='Effect of Unemployment Rate on Vehicle Type and Sales'))

        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1), html.Div(children=R_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3), html.Div(children=R_chart4)], style={'display': 'flex'})
        ]

    # Yearly Statistic Report Plots 
    # Check for Yearly Statistics.
    elif (input_year and selected_statistics == 'Yearly Statistics'):
        yearly_data = data[data['Year'] == int(input_year)]

        # Plot 1: Yearly Automobile sales using line chart for the whole period.
        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas, x='Year', y='Automobile_Sales',
                                            title='Yearly Automobile Sales'))

        # Plot 2: Total Monthly Automobile sales using line chart.
        mas = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(mas, x='Month', y='Automobile_Sales',
                                            title='Total Monthly Automobile Sales'))

        # Plot 3: bar chart for average number of vehicles sold during the given year
        avr_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(figure=px.bar(avr_vdata, x='Vehicle_Type', y='Automobile_Sales',
                                           title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year)))

        # Plot 4: Total Advertisement Expenditure for each vehicle using pie chart
        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(figure=px.pie(exp_data, values='Advertising_Expenditure', names='Vehicle_Type',
                                           title='Total Advertisement Expenditure for Each Vehicle'))

        return [
            html.Div(className='chart-item', children=[html.Div(children=Y_chart1), html.Div(children=Y_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=Y_chart3), html.Div(children=Y_chart4)], style={'display': 'flex'})
        ]
    else:
        return None



In [54]:
# Stop the previous Dash APP - kill the process using port 8050
!lsof -ti:8050 | xargs kill -9

In [56]:
# Run the Dash app
if __name__ == '__main__':
    app.run()